In [1]:
from config.settings import SRC_DIR, DATA_DIR
import os
# Define file path for the directory containing PDFs
file_dir = DATA_DIR / 'invoices' / 'valid_invoices'
files = os.listdir(file_dir)

files

['invoice.pdf',
 'ISS.pdf',
 'NEWFO-INV-7478.pdf',
 'output',
 'PerfectMatch.pdf',
 'PO139090 GP.pdf',
 'SKonica Sit23112709230.pdf',
 'skonica3-4.pdf']

In [2]:
file_path = file_dir/files[-1]
output_path = "C:/Users/ai.users/imscanCS/data/invoices/valid_invoices/output"
import fitz  # pymupdf

doc = fitz.open(str(file_path))
for i, page in enumerate(doc):
    pix = page.get_pixmap()
    pix.save(f"{output_path}/page_{i}.png") 

In [3]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import BitsAndBytesConfig

# Define the model name
model_name = "Qwen/Qwen2.5-VL-7B-Instruct"

# Configure 8-bit loading
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit precision
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="cuda"  # Automatically assigns layers to available devices
)

# Load the processor
processor = AutoProcessor.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
from config.invoice_config import get_qwen_prompt
# Messages containing multiple images and a text query
image_path_0 = 'C:/Users/ai.users/imscanCS/data/invoices/valid_invoices/output/page_0.png'
image_path_1 = 'C:/Users/ai.users/imscanCS/data/invoices/valid_invoices/output/page_1.png'
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path_0},
            {"type": "image", "image": image_path_1},
            # {"type": "image", "image": "file:///path/to/image2.jpg"},
            {"type": "text", "text": get_qwen_prompt()},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=4000)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['```json\n{\n    "line_items": [\n        {\n            "product_code": "8501025580",\n            "description": "FRA CAR 18X250 GEN VD DES K62H40 NG PAP",\n            "quantity": "327,00",\n            "price_per_unit": "5,886,00",\n            "vat_percent": "0,00 %",\n            "total_price": "1,471,50"\n        },\n        {\n            "product_code": "8501025581",\n            "description": "FRA CAR 18X250 GEN VD DES K62H40 NG PAP",\n            "quantity": "130,00",\n            "price_per_unit": "2,340,00",\n            "vat_percent": "0,00 %",\n            "total_price": "0,585,00"\n        },\n        {\n            "product_code": "8501025585",\n            "description": "FRA CAR 18X250 GEN VD DES K62H40 NG PAP",\n            "quantity": "170,00",\n            "price_per_unit": "3,095,00",\n            "vat_percent": "0,00 %",\n            "total_price": "0,765,00"\n        },\n        {\n            "product_code": "8501025611",\n            "description": "FRA CAR

In [5]:
import re, json
def extract(response_text):
    '''Extracts JSON data from model response.'''
    match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if match:
        extracted_data = json.loads(match.group(0))
        return extracted_data
    raise ValueError("No valid JSON found in response")

result = extract(output_text[0])

from IPython.display import display, HTML
from json2html import json2html

html_response = json2html.convert(json=result)
display(HTML(html_response))

In [18]:
from config.settings import ROOT_DIR
import os
output_folder = ROOT_DIR / 'invoice_outputs'/ model_name.split('/')[-1]
output_folder.mkdir(exist_ok=True)
file_name = str(file_path).split('/')[-1][:-4]
import json
# Define full file path
json_file_path = output_folder / f"{file_name}.json"

# Save JSON file
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(result, json_file, indent=4)

print(f"JSON saved at: {json_file_path}")

JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\Qwen2.5-VL-7B-Instruct\PerfectMatch.json
